## Import libraries

In [9]:
import pandas as pd
import numpy as np
import json
import requests
import random
from configparser import ConfigParser

## Credentials config parser

In [14]:
parser = ConfigParser()
_ = parser.read('credentials.cfg')

username = parser.get('monant_credentials', "username")
password = parser.get('monant_credentials', "password")

## Get access_token

In [12]:
url = 'https://api.monant.kinit.sk/auth'

credentials = {
    "username": username,
    "password": password
}

headers = {'content-type': 'application/json'
}

x = requests.post(url, data = json.dumps(credentials), headers=headers)

token = x.text
token = token.split(":")
token = token[1].replace('}', '')
token = token.replace('\"', '')

## Get .csv file with articles.ids

In [ ]:
value = 0

data = pd.DataFrame()
data_tmp = pd.DataFrame()
for i in range(0,7):
    print(i)
    query = {'monitoring_tags':'slovak',
             'published_at_from':'2018-01-01',
             'published_at_to':'2021-12-31',
             'count':'50000',
             'last_id':'{}'.format(value),
             'id_only':'true'}
    
    headers = {'content-type': 'application/json',
               'Accept-Charset': 'UTF-8',
               'Authorization': 'JWT {}'.format(access_token)}
    response = 'https://api.monant.kinit.sk/v1/articles'
    response = requests.get(response, params=query, headers=headers)

    data_tmp = response.json() 
    
    value = data_tmp['ids'][-1]
    
    data = data.append(data_tmp['ids'])
        

print(data)
data.columns=["ids"]
data.to_csv('sources/articles_id.csv', sep='\t', encoding='utf-8', index=False)

## Shuffle ids and get n articles

In [ ]:
articles = pd.read_csv('sources/articles_id.csv')
articles = articles['ids'].tolist()

random.shuffle(articles)

data = pd.DataFrame()
data_tmp = pd.DataFrame()
val = 0

articles = articles[:20000]
print(len(articles))
for article_id in articles:   
    headers = {'content-type': 'application/json',
               'Accept-Charset': 'UTF-8',
               'Authorization': 'JWT {}'.format(access_token)}
    response = 'https://api.monant.kinit.sk/v1/articles/{}'.format(article_id)
    response = requests.get(response, headers=headers)
    print(response)
    val = val + 1
    print(val)

    data_tmp = pd.json_normalize(response.json())
    data = data.append(data_tmp)       

print(data)

## Pickle dataframe 

In [ ]:
data.to_pickle("sources/df_articles.pkl")